In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.error_rate_mu = 0.0
        self.error_rate_sigma = 0.0

    def set_error_rate(self, mu, sigma):
        self.error_rate_mu = mu
        self.error_rate_sigma = sigma
    
    def apply_error_rate(self, x):
        if self.error_rate_mu > 0 or self.error_rate_sigma > 0:
            noise = torch.normal(mean=self.error_rate_mu, std=self.error_rate_sigma, size=x.size())
            noise = torch.abs(noise)
            x = x + noise*x
        return x

    def forward(self, x):
        out = self.fc1(x)
        out = self.apply_error_rate(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.apply_error_rate(out)
        return out

In [2]:
# Hyperparameters
input_size = 28 * 28 # Fashion MNIST images are 28x28 pixels
hidden_size = 256
num_classes = 10 # Fashion MNIST has 10 classes
num_epochs = 50
batch_size = 100
learning_rate = 0.001

# Load the Fashion MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [3]:
# Initialize the model, loss function, and optimizer
model = MLP(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# Training the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Flatten images to [batch_size, input_size]
        images = images.reshape(-1, 28 * 28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'\r Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item():.4f}, Accuracy: {100 * (labels == outputs.argmax(1)).sum().item() / batch_size:.2f}%')

 Epoch [1/50], Step [100/600], Loss: 0.3341, Accuracy: 90.00%
 Epoch [1/50], Step [200/600], Loss: 0.5838, Accuracy: 77.00%
 Epoch [1/50], Step [300/600], Loss: 0.4617, Accuracy: 84.00%
 Epoch [1/50], Step [400/600], Loss: 0.4090, Accuracy: 85.00%
 Epoch [1/50], Step [500/600], Loss: 0.5551, Accuracy: 77.00%
 Epoch [1/50], Step [600/600], Loss: 0.4294, Accuracy: 82.00%
 Epoch [2/50], Step [100/600], Loss: 0.4042, Accuracy: 85.00%
 Epoch [2/50], Step [200/600], Loss: 0.3996, Accuracy: 86.00%
 Epoch [2/50], Step [300/600], Loss: 0.4558, Accuracy: 84.00%
 Epoch [2/50], Step [400/600], Loss: 0.3061, Accuracy: 87.00%
 Epoch [2/50], Step [500/600], Loss: 0.3739, Accuracy: 85.00%
 Epoch [2/50], Step [600/600], Loss: 0.3538, Accuracy: 89.00%
 Epoch [3/50], Step [100/600], Loss: 0.1780, Accuracy: 95.00%
 Epoch [3/50], Step [200/600], Loss: 0.4358, Accuracy: 85.00%
 Epoch [3/50], Step [300/600], Loss: 0.3195, Accuracy: 92.00%
 Epoch [3/50], Step [400/600], Loss: 0.3202, Accuracy: 89.00%
 Epoch [

In [7]:
# Testing the model
model.eval() # Set the model to evaluation mode
for with_error in [False, True]:
    if with_error:
        model.set_error_rate(0.1497, 0.2735)
    else:
        model.set_error_rate(0.0, 0.0)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.reshape(-1, 28 * 28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        if not with_error:
            print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')
        else:
            print(f'Accuracy of the model with error on the 10000 test images: {100 * correct / total:.2f}%')

    # Save the model checkpoint
    torch.save(model.state_dict(), 'mlp_fashion_mnist.pth')

Accuracy of the model on the 10000 test images: 88.72%
Accuracy of the model with error on the 10000 test images: 88.06%
